# Sentiment Analysis
> * Positive or Negative

In [ ]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re

from collections import namedtuple
from gensim.models import doc2vec
import multiprocessing
from konlpy.tag import Mecab
from konlpy.utils import pprint
import numpy as np
from ckonlpy.tag import Twitter as ctwitter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
mecab = Mecab()
ct = ctwitter()

## Doc2Vec

In [ ]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags')

#### twitter

In [ ]:
train = pickle.load(open('./data/pre_data/pre_by_ct_train.pickled','rb'))
test = pickle.load(open('./data/pre_data/pre_by_ct_test.pickled','rb'))

In [ ]:
model = doc2vec.Doc2Vec.load('./model/doc2vec_size2000_epoch30_by_ct.model')

In [ ]:
train_x = [model.infer_vector(doc.words) for doc in train]
train_y = [doc.tags[0] for doc in train]
test_x = [model.infer_vector(doc.words) for doc in test]
test_y = [doc.tags[0] for doc in test]

In [ ]:
classifier = LogisticRegression(random_state=1234)
classifier.fit(train_x, train_y)

In [ ]:
print( classifier.score(test_x, test_y) )

#### mecab

In [ ]:
train2 = pickle.load(open('./data/pre_data/pre_by_mecab_train.pickled','rb'))
test2 = pickle.load(open('./data/pre_data/pre_by_mecab_test.pickled','rb'))

In [ ]:
model2 = doc2vec.Doc2Vec.load('./model/doc2vec_size2000_epoch30_by_mecab.model')

In [ ]:
train2_x = [model2.infer_vector(doc.words) for doc in train2]
train2_y = [doc.tags[0] for doc in train2]
test2_x = [model2.infer_vector(doc.words) for doc in test2]
test2_y = [doc.tags[0] for doc in test2]

In [ ]:
classifier2 = LogisticRegression(random_state=1234)
classifier2.fit(train2_x, train2_y)

In [ ]:
print( classifier2.score(test2_x, test2_y) )